<a href="https://colab.research.google.com/github/Saint-Pedro/M1-Qualite-des-donnees-EPSI-TP2/blob/main/Qualite_des_donnes_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- BLOC 1 : Installation et Structure ---
!pip install "great_expectations==0.18.19" pandas numpy pyarrow fastparquet

import os
import logging
import sys
import pandas as pd
import numpy as np

# Création structure
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("src", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)


In [17]:
# --- BLOC 2 ---
import pandas as pd
import numpy as np
import logging

# Configuration logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Chemin du fichier
INPUT_FILE = "data/raw/food_sample.parquet"

# --- 1. Chargement ---
try:
    df_raw = pd.read_parquet(INPUT_FILE)
    logger.info(f"Dataset chargé : {df_raw.shape[0]} lignes, {df_raw.shape[1]} colonnes")
except FileNotFoundError:
    logger.error("Fichier non trouvé ! Vérifiez le dossier data/raw/")
    raise

# --- 2. Sélection des variables ---
# Liste cible idéale
cols_target = [
    'code', 'product_name', 'brands', 'categories',
    'nutriscore_grade', 'nutriscore_score',
    'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g',
    'sugars_100g', 'proteins_100g', 'salt_100g', 'carbohydrates_100g'
]

# On ne garde que ce qui existe VRAIMENT dans le fichier
existing_cols = [c for c in cols_target if c in df_raw.columns]
df_selected = df_raw[existing_cols].copy()

# Affichage des infos techniques
print("--- Infos techniques ---")
df_selected.info()

# [cite_start]--- 3. Dictionnaire de données (Correction dynamique) --- [cite: 171]

# On crée un "mapping" (dictionnaire) des définitions pour éviter l'erreur de longueur
definitions_map = {
    'code': "Code barre (ID)",
    'product_name': "Nom du produit",
    'brands': "Marque",
    'categories': "Catégorie",
    'nutriscore_grade': "Grade Nutri-Score (A-E)",
    'nutriscore_score': "Score numérique",
    'energy-kcal_100g': "Calories (kcal)",
    'fat_100g': "Graisses",
    'saturated-fat_100g': "Graisses saturées",
    'sugars_100g': "Sucres",
    'proteins_100g': "Protéines",
    'salt_100g': "Sel",
    'carbohydrates_100g': "Glucides"
}

# On construit le tableau en allant chercher la définition seulement si la colonne existe
data_dict = pd.DataFrame({
    "Variable": existing_cols,
    "Type Python": [str(df_selected[c].dtype) for c in existing_cols],
    "Exemple": [df_selected[c].iloc[0] if not df_selected.empty else "" for c in existing_cols],
    "Définition": [definitions_map.get(c, "Non défini") for c in existing_cols]
})

print("\n--- Dictionnaire des données ---")
display(data_dict)

  return datetime.utcnow().replace(tzinfo=utc)



--- Infos techniques ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119532 entries, 0 to 119531
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   code              119532 non-null  object 
 1   product_name      119532 non-null  object 
 2   brands            69501 non-null   object 
 3   categories        64957 non-null   object 
 4   nutriscore_grade  116758 non-null  object 
 5   nutriscore_score  45347 non-null   float64
dtypes: float64(1), object(5)
memory usage: 5.5+ MB

--- Dictionnaire des données ---


,Variable,Type Python,Exemple,Définition
0,code,object,0000101209159,Code barre (ID)
1,product_name,object,"[{'lang': 'main', 'text': 'Véritable pâte à ta...",Nom du produit
2,brands,object,Bovetti,Marque
3,categories,object,"Petit-déjeuners,Produits à tartiner,Produits à...",Catégorie
4,nutriscore_grade,object,e,Grade Nutri-Score (A-E)
5,nutriscore_score,float64,25.0,Score numérique


In [18]:
# --- BLOC 3 : Audit Qualité  ---
import great_expectations as gx

print(f"Version GX utilisée : {gx.__version__}")

# 1. Initialisation Contexte
context = gx.get_context(mode="ephemeral")

# 2. Création Source & Asset (Syntaxe GX 0.18)
datasource = context.sources.add_pandas(name="off_datasource")
# On utilise df_selected (créé au Bloc 2)
asset = datasource.add_dataframe_asset(name="raw_data", dataframe=df_selected)

# 3. Suite de règles
suite_name = "audit_raw_data"
context.add_or_update_expectation_suite(expectation_suite_name=suite_name)

batch_request = asset.build_batch_request()
validator = context.get_validator(batch_request=batch_request, expectation_suite_name=suite_name)

# --- Définition des Règles ---

# A. Complétude
if 'product_name' in df_selected.columns:
    validator.expect_column_values_to_not_be_null(column="product_name")

if 'brands' in df_selected.columns:
    validator.expect_column_values_to_not_be_null(column="brands")

# B. Unicité
if 'code' in df_selected.columns:
    validator.expect_column_values_to_be_unique(column="code")

# C. Validité & D. Conformité
# On vérifie l'existence des colonnes avant d'ajouter les règles
if 'nutriscore_grade' in df_selected.columns:
    validator.expect_column_values_to_be_in_set(column="nutriscore_grade", value_set=['a', 'b', 'c', 'd', 'e'])

if 'code' in df_selected.columns:
    validator.expect_column_values_to_match_regex(column="code", regex=r"^\d+$")

# Règles sur les nutriments
nutri_cols_check = ['sugars_100g', 'fat_100g']
for col in nutri_cols_check:
    if col in df_selected.columns:
        validator.expect_column_values_to_be_between(column=col, min_value=0, max_value=100)
    else:
        logger.warning(f"Règle ignorée : la colonne '{col}' n'existe pas dans le dataset.")

# 4. Exécution
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint = context.add_or_update_checkpoint(name="raw_checkpoint", validator=validator)
results = checkpoint.run()

# 5. Résultats
stats = results["run_results"][list(results["run_results"].keys())[0]]["validation_result"]["statistics"]
print(f"\n--- RÉSULTAT AUDIT ---")
print(f"Taux de succès : {stats['success_percent']:.2f}%")
print(f"Règles validées : {stats['successful_expectations']} / {stats['evaluated_expectations']}")

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp_61mc6nj' for ephemeral docs site
  return datetime.utcnow().replace(tzinfo=utc)



Version GX utilisée : 0.18.19


  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  warnings.warn(



...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  return datetime.utcnow().replace(tzinfo=utc)

  return datetime.utcnow().replace(tzinfo=utc)



Calculating Metrics:   0%|          | 0/32 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  return datetime.utcnow().replace(tzinfo=utc)




--- RÉSULTAT AUDIT ---
Taux de succès : 40.00%
Règles validées : 2 / 5


In [19]:
# --- BLOC 4 : Traitement et Nettoyage ---

df_clean = df_selected.copy()
logger.info("Début du nettoyage...")

# 1. Création des colonnes manquantes
target_cols = ['energy-kcal_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'proteins_100g', 'salt_100g', 'carbohydrates_100g']
for col in target_cols:
    if col not in df_clean.columns:
        df_clean[col] = np.nan # On crée la colonne vide

# 2. Dédoublonnage
if 'code' in df_clean.columns:
    initial_len = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['code'], keep='first')
    logger.info(f"Dédoublonnage : {initial_len - len(df_clean)} doublons supprimés.")

# 3. Nettoyage Types Numériques
def clean_numeric(val):
    if pd.isna(val): return np.nan
    if isinstance(val, (int, float)): return float(val)
    import re
    val_str = str(val).replace(',', '.')
    match = re.search(r"([\d\.]+)", val_str)
    return float(match.group(1)) if match else np.nan

for col in target_cols:
    df_clean[col] = df_clean[col].apply(clean_numeric)

# 4. Standardisation Texte
if 'nutriscore_grade' in df_clean.columns:
    df_clean['nutriscore_grade'] = df_clean['nutriscore_grade'].astype(str).str.lower()

# Sauvegarde
OUTPUT_FILE = "data/processed/food_sample_clean.parquet"
df_clean.to_parquet(OUTPUT_FILE)
logger.info(f"Fichier propre sauvegardé : {OUTPUT_FILE}")
display(df_clean.head(3))

  return datetime.utcnow().replace(tzinfo=utc)



,code,product_name,brands,categories,nutriscore_grade,nutriscore_score,energy-kcal_100g,fat_100g,saturated-fat_100g,sugars_100g,proteins_100g,salt_100g,carbohydrates_100g
0,0000101209159,"[{'lang': 'main', 'text': 'Véritable pâte à ta...",Bovetti,"Petit-déjeuners,Produits à tartiner,Produits à...",e,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000260440264,"[{'lang': 'main', 'text': 'Confit d'Oignons ou...",None,"Aliments et boissons à base de végétaux, Alime...",d,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000682009841,"[{'lang': 'main', 'text': 'Pain de campagne bi...",,Pains de campagne,c,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# --- BLOC 5 : Audit sur données propres ---

# Nouvelle source pour les données propres
datasource_clean = context.sources.add_pandas(name="clean_datasource")
asset_clean = datasource_clean.add_dataframe_asset(name="clean_data", dataframe=df_clean)

# On reprend la même suite de règles
batch_request_clean = asset_clean.build_batch_request()
validator_clean = context.get_validator(batch_request=batch_request_clean, expectation_suite_name=suite_name)

# On force l'ajout des règles sur les colonnes qui manquaient avant mais existent maintenant
validator_clean.expect_column_values_to_be_between(column="sugars_100g", min_value=0, max_value=100)

checkpoint_clean = context.add_or_update_checkpoint(name="clean_checkpoint", validator=validator_clean)
results_clean = checkpoint_clean.run()

stats_clean = results_clean["run_results"][list(results_clean["run_results"].keys())[0]]["validation_result"]["statistics"]

print(f"--- COMPARATIF ---")
print(f"Après nettoyage : {stats_clean['success_percent']:.2f}% de succès")

  return datetime.utcnow().replace(tzinfo=utc)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  return datetime.utcnow().replace(tzinfo=utc)



Calculating Metrics:   0%|          | 0/32 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  return datetime.utcnow().replace(tzinfo=utc)



--- COMPARATIF ---
Après nettoyage : 60.00% de succès


In [21]:
# --- BLOC 6 : Analyse Métier ---

# Règle 1 : Energie aberrante (> 1000 kcal)
# On remplit les NaN par 0 pour pouvoir faire le calcul de filtre sans erreur
mask_aberrant_energy = df_clean['energy-kcal_100g'].fillna(0) > 1000

# Règle 2 : Somme > 100g
df_clean['sum_macro'] = (df_clean['fat_100g'].fillna(0) +
                         df_clean['carbohydrates_100g'].fillna(0) +
                         df_clean['proteins_100g'].fillna(0) +
                         df_clean['salt_100g'].fillna(0))
mask_aberrant_sum = df_clean['sum_macro'] > 110

print(f"Produits > 1000 kcal : {mask_aberrant_energy.sum()}")
print(f"Produits somme > 110g : {mask_aberrant_sum.sum()}")

# Exclusion
df_final = df_clean[~mask_aberrant_energy & ~mask_aberrant_sum].copy()
print(f"Taille finale du dataset : {len(df_final)}")

  return datetime.utcnow().replace(tzinfo=utc)



Produits > 1000 kcal : 0
Produits somme > 110g : 0
Taille finale du dataset : 119531


In [22]:
print("""
### MONITORING RECOMMANDÉ
1. Taux de complétude 'nutriscore_grade' (Alerte si < 90%)
2. Taux de validité Kcal (Alerte si > 1% de valeurs > 900)
3. Unicité Code Barre (Bloquant si doublon)
""")


### MONITORING RECOMMANDÉ
1. Taux de complétude 'nutriscore_grade' (Alerte si < 90%)
2. Taux de validité Kcal (Alerte si > 1% de valeurs > 900)
3. Unicité Code Barre (Bloquant si doublon)

